# A. Load HyP3 SBAS Stack into MintPy

*Author: Alex Lewandowski; Alaska Satellite Facility*

**Load your previously downloaded HyP3 SBAS stack into MintPy.**

- Load an SBAS stack from many GeoTiffs into two HDF5 datasets
  - `geometryGeo.h5`
  - `ifgramStack.h5`
 
You only need to perform this step once. You will be able to configure and reconfigure the parameters of your time series analysis without having to reload the data.

This notebook will provide the option to delete your GeoTiffs once they are loaded by MintPy

---

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div>
    <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">
      <img src="github_issues.png" alt="GitHub logo over the word Issues" style="width: 75px; margin-right: 10px;">
    </a>
  </div>
  <div>
    <b>Find a notebook bug or have a feature request?</b>
    <br/>
    Explore current issues on its GitHub repository, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">opensarlab_MintPy_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div>
    <a href="mailto:uso@asf.alaska.edu">
      <img src="ASF_support_logo.png" alt="ASF logo" style="width: 75px; margin-right: 10px;">
    </a>
  </div>
  <div>
    <b>Have a question related to SAR, ASF data access, or performing SBAS time series analyses with MintPy?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---

## 0. Import Required Software

In [ ]:
from pathlib import Path
from pprint import pprint
import shutil

import h5py
from ipyfilechooser import FileChooser
from mintpy.cli import plot_network
import opensarlab_lib as osl

---
## 1. Create a Custom Config File Defining the Paths to the GeoTiffs

**Select the directory holding your MintPy-ready HyP3 data stack and/or MintPy directory from a previously loaded MintPy SBAS stack**
- Click the `Select` button
- Navigate to your data directory
- Click the `Select` button
- Confirm that the desired path appears in green text
- Click the `Change` button to alter your selection

In [ ]:
path = Path.home()
fc = FileChooser(path)
display(fc)

**Define a project name and create a MintPy directory in which to store files output during our analysis**

In [ ]:
# define the work directory
data_path = Path(fc.selected_path)
print(f"Work directory: {data_path}")

# define the MintPy time-series directory
mint_path = data_path / 'MintPy'
mint_path.mkdir(exist_ok=True)
print(f"MintPy directory: {mint_path}")

# define a project name
proj_name = input("Enter a project name: ")
custom_config_path = mint_path / f'{proj_name}.txt'
print(f"Custom config path: {custom_config_path}")

#create a directory in which to store plots
plot_path = mint_path / "plots"
plot_path.mkdir(exist_ok=True)

**Define the paths to the GeoTiffs in a custom config**

In [ ]:
con_comp_path = list(data_path.glob('./*/*_conncomp*.tif'))
burst = len(con_comp_path) > 0

config = f"""
mintpy.load.processor        = hyp3
##---------geometry datasets:
mintpy.load.demFile          = {data_path}/*/*_dem*.tif
mintpy.load.incAngleFile     = {data_path}/*/*_lv_theta*.tif
mintpy.load.azAngleFile      = {data_path}/*/*_lv_phi*.tif
mintpy.load.waterMaskFile    = {data_path}/*/*_water_mask*.tif
##---------interferogram datasets:
mintpy.load.unwFile          = {data_path}/*/*_unw_phase*.tif
mintpy.load.corFile          = {data_path}/*/*_corr*.tif
"""

if burst:
    config += f"mintpy.load.connCompFile     = {data_path}/*/*_conncomp*.tif"
    
print(config)
custom_config_path.write_text(config)

---
## 2. Load the Data with MintPy

The output of the loading step is saved to the `MintPy/inputs` directory:
- ifgramStack.h5: 
- geometryGeo.h5

In [ ]:
!smallbaselineApp.py $custom_config_path --work-dir {mint_path} --dostep load_data

In [ ]:
inputs_path = mint_path/'inputs'
!ls $inputs_path

---
## 3. Run the Modify Network Step

- Identifies and excludes interferograms (i.e. affected by coherence phase-unwrapping errors) before the network inversion

In [ ]:
!smallbaselineApp.py $custom_config_path --work-dir {mint_path} --dostep modify_network

---
## 4. Plot the interferogram network

Running **plot_network** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- `ifgramStack_coherence_spatialAvg.txt`: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- `Network.pdf`: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- `CoherenceMatrix.pdf` shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
with osl.work_dir(mint_path):
    scp_args = f'{inputs_path}/ifgramStack.h5 --cmap-vlist 0.2 0.7 1.0'
    plot_network.main(scp_args.split())
    plots = ['pbaseHistory.pdf', 'coherenceHistory.pdf', 'coherenceMatrix.pdf', 'network.pdf']
    for p in plots:
        if (mint_path/p).exists():
            (mint_path/p).rename(f'{plot_path}/{p}')

---
## 5. Delete Input Data

Once MintPy has loaded the data into HDF5 datasets, you may wish to delete the input data to conserve storage space. 

You can adjust configuration parameters and update your analysis without reloading the data from their original source.

In [ ]:
s1_paths = list(data_path.glob('S1*'))
pprint(f'S1 directories: {s1_paths}')

In [ ]:
cleanup_select = osl.select_parameter(
    [
        'Delete S1 directories',
        'Save S1 directories'
    ]
)
print("Save or delete input data:")
display(cleanup_select)

In [ ]:
if 'Delete' in cleanup_select.value:
    for d in s1_paths:
        shutil.rmtree(d)